In [ ]:
"""Notebook contains ICP registration of the two measurement point clouds,
along with their pre-processing steps.
"""

In [ ]:
import open3d as o3d
import numpy as np

from utils import threshold_pc_distance
from utils import visualizer_pc
from utils import visualize_two_pc

# Import, visualize, perform spatial filtering and transform

In [ ]:
pc_source = o3d.io.read_point_cloud("./measurements/pumpkin/20230717-190933.pcd")
pc_target = o3d.io.read_point_cloud("./measurements/pumpkin/20230717-190946.pcd")

In [ ]:
pc_source = threshold_pc_distance(pc_source, 250, 750)
pc_target = threshold_pc_distance(pc_target, 250, 750)

In [ ]:
# Rotate point clouds around Y and X axis, due to the coordinate system diference of OAK-D camera.
rotation = o3d.geometry.get_rotation_matrix_from_xyz((0, np.pi / 2, 0))
pc_source.rotate(rotation, center=(0, 0, 0))
pc_target.rotate(rotation, center=(0, 0, 0))

In [ ]:
vis = visualizer_pc(pc_source)
vis.show()

In [ ]:
vis = visualizer_pc(pc_target)
vis.show()

# Translate & estimate normals

In [ ]:
translation_vector = pc_source.get_center() - pc_target.get_center()
pc_target.translate(translation_vector)

In [ ]:
# Estimate pc normals.
radius_normal = 0.005  # 5 mm.
pc_source.estimate_normals(
    o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30)
)
pc_target.estimate_normals(
    o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30)
)

# Register the clouds

In [ ]:
loss = o3d.pipelines.registration.TukeyLoss(k=0.1)
result = o3d.pipelines.registration.registration_icp(
    pc_source,
    pc_target,
    np.identity(4),
    o3d.pipelines.registration.TransformationEstimationPointToPlane(loss),
)

In [ ]:
print(result)

In [ ]:
visualize_two_pc(pc_source, pc_target, 1.5, "Initial state")

In [ ]:
visualize_two_pc(pc_source, pc_target, 1.5, "ICP", result.transformation)